In [10]:
import spotipy
import os
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import requests
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# secret info
client_id = 'x'
client_secret = 'x'
redirect_uri='http://localhost:8000'
scope = 'user-library-read user-top-read user-read-recently-played playlist-modify-public'

# authenticate with Spotify API
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope))


# ChatGPT API credentials
chatgpt_api_key = 'x'
chatgpt_url = 'https://api.openai.com/v1/engines/davinci-codex/completions'

def get_track_recommendations_by_mood(mood, num_songs):
    """retrieve track recommendations based on a given mood using Spotify API."""
    try:
        # Use the Spotify API to get track recommendations based on mood
        results = sp.recommendations(seed_genres=[mood], limit=num_songs)
        # Extract the track URIs from the results
        tracks = [track['uri'] for track in results['tracks']]
        print(f"Retrieved {len(tracks)} tracks for {mood} mood.")
        return tracks
    except spotipy.SpotifyException as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None



def get_popular_playlists_by_mood(mood):
    """retrieve popular playlists based on a given mood using Spotify API."""
    try:
        # Use the Spotify API to search for popular playlists based on mood
        results = sp.search(q=mood, type='playlist', limit=5)
        # Extract the playlist IDs and names from the results
        playlists = [(playlist['id'], playlist['name']) for playlist in results['playlists']['items']]
        return playlists
    except spotipy.SpotifyException as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

def generate_playlist_name(mood, occasion):
    """generate a playlist name based on the mood and occasion."""
    try:
        # Use the ChatGPT API to generate a playlist name based on mood and occasion
        prompt = f"Generate a playlist name for {mood} {occasion} playlist"
        headers = {'Authorization': f'Bearer {chatgpt_api_key}'}
        data = {'prompt': prompt}
        response = requests.post(chatgpt_url, headers=headers, json=data)
        playlist_name = response.json()['choices'][0]['text']
        return playlist_name
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None
    
def create_playlist(mood, num_songs, occasion):
    """create a playlist based on the given mood, number of songs, and occasion."""
    # Validate the mood input
    if not mood:
        print("Error: Mood cannot be empty.")
        return {'error': 'Mood cannot be empty.'}
    # Retrieve track recommendations based on mood
    tracks = get_track_recommendations_by_mood(mood, num_songs)
    if not tracks:
        print(f"Error: Could not retrieve tracks for {mood} mood.")
        return {'error': f"Could not retrieve tracks for {mood} mood."}
    # Retrieve popular playlists based on mood
    playlists = get_popular_playlists_by_mood(mood)
    if not playlists:
        print(f"Error: Could not retrieve playlists for {mood} mood.")
        return {'error': f"Could not retrieve playlists for {mood} mood."}
    # Pick a playlist to use as a template
    playlist_id, playlist_name = playlists[0]
    # Create a new playlist using the template
    new_playlist_id = create_empty_playlist(playlist_name, occasion)
    if not new_playlist_id:
        print(f"Error: Could not create playlist for {occasion} occasion.")
        return {'error': f"Could not create playlist for {occasion} occasion."}
    # Add the recommended tracks to the new playlist
    if not add_tracks_to_playlist(new_playlist_id, tracks):
        print("Error: Could not add tracks to the playlist.")
        return {'error': "Could not add tracks to the playlist."}
    # Return the ID and tracks of the new playlist
    return {'id': new_playlist_id, 'name': playlist_name, 'tracks': tracks}


def create_empty_playlist(playlist_name, occasion):
    """create an empty playlist with a given name and description using Spotify API."""
    try:
        # Get the user ID
        user_id = sp.me()['id']
        # Create the empty playlist with the given name and description
        playlist = sp.user_playlist_create(user_id, playlist_name, description=f"{occasion} playlist")
        # Extract the playlist ID from the response
        playlist_id = playlist['id']
        print(f"Created playlist '{playlist_name}' with ID '{playlist_id}'.")
        return playlist_id
    except spotipy.SpotifyException as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

def add_tracks_to_playlist(playlist_id, tracks):
    """add a list of tracks to a given playlist using Spotify API."""
    try:
        # Add the tracks to the playlist
        sp.user_playlist_add_tracks(sp.me()['id'], playlist_id, tracks)
        print(f"Added {len(tracks)} tracks to playlist '{playlist_id}'.")
        return True
    except spotipy.SpotifyException as e:
        print(f"Error: {e}")
        return False
    except Exception as e:
        print(f"Error: {e}")
        return False

def main():
    """main function to generate a playlist based on user input."""
    # Prompt the user for input
    mood = input("Enter a mood for your playlist: ")
    num_songs = int(input("Enter the number of songs you want in your playlist: "))
    occasion = input("Enter the occasion for your playlist (e.g. party, workout, study): ")
    # Generate a playlist name using the ChatGPT API
    playlist_name = generate_playlist_name(mood, occasion)
    print(f"Generated playlist name: {playlist_name}")
    # Create a playlist using Spotify API
    playlist_info = create_playlist(mood, num_songs, occasion)
    if 'error' in playlist_info:
        print(f"Error: {playlist_info['error']}")
    else:
        # Print the playlist information
        print(f"Created playlist '{playlist_info['name']}' with ID '{playlist_info['id']}' and {len(playlist_info['tracks'])} tracks.")

In [17]:
if __name__ == '__main__':
    main()

Enter a mood for your playlist: romance
Enter the number of songs you want in your playlist: 10
Enter the occasion for your playlist (e.g. party, workout, study): date
Generated playlist name: "
        playlist_name = 'Romance Date - %s' %
Retrieved 10 tracks for romance mood.
Created playlist 'Romantic Mix' with ID '4IHPl0cSwQgLaBogVDUQXS'.
Added 10 tracks to playlist '4IHPl0cSwQgLaBogVDUQXS'.
Created playlist 'Romantic Mix' with ID '4IHPl0cSwQgLaBogVDUQXS' and 10 tracks.
